# T1176 - Browser Extensions
Adversaries may abuse Internet browser extensions to establish persistence access to victim systems. Browser extensions or plugins are small programs that can add functionality and customize aspects of Internet browsers. They can be installed directly or through a browser's app store and generally have access and permissions to everything that the browser can access. (Citation: Wikipedia Browser Extension) (Citation: Chrome Extensions Definition)

Malicious extensions can be installed into a browser through malicious app store downloads masquerading as legitimate extensions, through social engineering, or by an adversary that has already compromised a system. Security can be limited on browser app stores so it may not be difficult for malicious extensions to defeat automated scanners. (Citation: Malicious Chrome Extension Numbers) Once the extension is installed, it can browse to websites in the background, (Citation: Chrome Extension Crypto Miner) (Citation: ICEBRG Chrome Extensions) steal all information that a user enters into a browser (including credentials) (Citation: Banker Google Chrome Extension Steals Creds) (Citation: Catch All Chrome Extension) and be used as an installer for a RAT for persistence.

There have also been instances of botnets using a persistent backdoor through malicious Chrome extensions. (Citation: Stantinko Botnet) There have also been similar examples of extensions being used for command & control  (Citation: Chrome Extension C2 Malware).

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Chrome (Developer Mode)
Turn on Chrome developer mode and Load Extension found in the src directory
**Supported Platforms:** linux, windows, macos
Run it with these steps!
1. Navigate to [chrome://extensions](chrome://extensions) and
tick 'Developer Mode'.

2. Click 'Load unpacked extension...' and navigate to
[Browser_Extension](../t1176/src/)

3. Click 'Select'


### Atomic Test #2 - Chrome (Chrome Web Store)
Install the "Minimum Viable Malicious Extension" Chrome extension
**Supported Platforms:** linux, windows, macos
Run it with these steps!
1. Navigate to https://chrome.google.com/webstore/detail/minimum-viable-malicious/odlpfdolehmhciiebahbpnaopneicend
in Chrome

2. Click 'Add to Chrome'


### Atomic Test #3 - Firefox
Create a file called test.wma, with the duration of 30 seconds

**Supported Platforms:** linux, windows, macos
Run it with these steps!
1. Navigate to [about:debugging](about:debugging) and
click "Load Temporary Add-on"

2. Navigate to [manifest.json](./src/manifest.json)

3. Then click 'Open'


### Atomic Test #4 - Edge Chromium Addon - VPN
Adversaries may use VPN extensions in an attempt to hide traffic sent from a compromised host. This will install one (of many) available VPNS in the Edge add-on store.

**Supported Platforms:** windows, macos
Run it with these steps!
1. Navigate to https://microsoftedge.microsoft.com/addons/detail/fjnehcbecaggobjholekjijaaekbnlgj
in Edge Chromium

2. Click 'Get'


## Detection
Inventory and monitor browser extension installations that deviate from normal, expected, and benign extensions. Process and network monitoring can be used to detect browsers communicating with a C2 server. However, this may prove to be a difficult way of initially detecting a malicious extension depending on the nature and volume of the traffic it generates.

Monitor for any new items written to the Registry or PE files written to disk. That may correlate with browser extension installation.

# Shield Active Defense
## Baseline 
 Identify key system elements to establish a baseline and be prepared to reset a system to that baseline when necessary. 

 Identify elements of software and configuration critical to a set of objectives, define their proper values, and be prepared to reset a running system to its intended state.  
### Opportunity
There is an opportunity to use tools and controls to stop an adversary's activity.
### Use Case
A defender can force the removal of browser extensions that are not allowed by a corporate policy.
### Procedures
Maintain a verified baseline firewall configuration and use that copy as a fallback if an adversary alters that information.
Maintain a verified list of group policies enforced on a system and use that copy if an adversary attempts to deviate from the baseline.